In [1]:
import cl_graph_bert as cgm
import torch
from torch import nn
import json
from transformers import BertModel, BertConfig, BertTokenizer, AdamW
import tqdm

from torch.utils.data import Dataset, DataLoader

import dgl
g = dgl.load_graphs("./graphs/industrial_and_scientific_5_core_new.dgl")[0][0]

In [2]:
model = cgm.CLIPGraphModel(
    rel_types = g.etypes,
    emb_types = {x: g.number_of_nodes(x) for x in g.ntypes} 
)

device = "cuda"

model.load_state_dict(torch.load("./base_statedict_6668.27483537762.pt", map_location=torch.device('cuda')))
model.eval()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CLIPGraphModel(
  (graph_model): RGCN(
    (embed): ParameterDict(
        (Brand): Parameter containing: [torch.FloatTensor of size 1900x512]
        (Customer): Parameter containing: [torch.FloatTensor of size 11041x512]
        (Product): Parameter containing: [torch.FloatTensor of size 5334x512]
        (Review): Parameter containing: [torch.FloatTensor of size 77060x512]
    )
    (conv1): HeteroGraphConv(
      (mods): ModuleDict(
        (rev_SOLD_BY): GraphConv(in=512, out=256, normalization=both, activation=None)
        (WROTE): GraphConv(in=512, out=256, normalization=both, activation=None)
        (SOLD_BY): GraphConv(in=512, out=256, normalization=both, activation=None)
        (rev_REVIEW_OF): GraphConv(in=512, out=256, normalization=both, activation=None)
        (REVIEW_OF): GraphConv(in=512, out=256, normalization=both, activation=None)
        (rev_WROTE): GraphConv(in=512, out=256, normalization=both, activation=None)
      )
    )
    (conv2): HeteroGraphConv(
     

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
import tqdm

bert_base = BertModel.from_pretrained('bert-base-uncased')
device = "cuda"

bert_base.to(device)
model.language_model.to(device)
model.graph_model.to(device)
g.to(device)

reivew_ids = []

gnn_sims = []
bert_base_sims = []
bert_finetuned_sims = []

with torch.no_grad():

    for i in tqdm.tqdm(range(g.num_nodes('Product'))):
        review_id = g.successors(i, 'rev_REVIEW_OF').to(device)
        
        if len(review_id) > 1 and len(review_id) < 650:

            reivew_ids.append(review_id)
            
            gnn_embs = model.graph_model.embed["Review"][review_id].detach().cpu().numpy()
            
            base_bert_embs = bert_base(input_ids = g.nodes["Review"].data['input_ids'][review_id].to(device), 
               attention_mask=g.nodes["Review"].data['attention_mask'][review_id].to(device), 
               token_type_ids=g.nodes["Review"].data['token_type_ids'][review_id].to(device)).last_hidden_state[:,0].type(torch.float64).detach().cpu().numpy()
            
            bert_finetuned_embs = model.language_model(input_ids = g.nodes["Review"].data['input_ids'][review_id].to(device), 
               attention_mask=g.nodes["Review"].data['attention_mask'][review_id].to(device), 
               token_type_ids=g.nodes["Review"].data['token_type_ids'][review_id].to(device)).last_hidden_state[:,0].type(torch.float64).detach().cpu().numpy()

            gnn_sims.append( (np.sum(cosine_similarity(gnn_embs)) - len(review_id)) /  (len(review_id) ** 2 - len(review_id)) )
            bert_base_sims.append( (np.sum(cosine_similarity(base_bert_embs)) - len(review_id)) /  (len(review_id) ** 2 - len(review_id)) )
            bert_finetuned_sims.append( (np.sum(cosine_similarity(bert_finetuned_embs)) - len(review_id)) / (len(review_id) ** 2 - len(review_id)) )
            print(bert_finetuned_sims[-1])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|                                                                                | 1/5334 [00:00<22:08,  4.01it/s]

0.9982072246204989


  0%|                                                                                | 2/5334 [00:00<26:29,  3.36it/s]

0.9979733548217031


  0%|                                                                                | 3/5334 [00:01<37:29,  2.37it/s]

...

0.9974633767277408
0.9974066552989402
0.9978573985132613


  0%|▎                                                                              | 20/5334 [00:05<18:10,  4.87it/s]

0.9977119950443676


  0%|▎                                                                              | 20/5334 [00:05<26:22,  3.36it/s]


KeyboardInterrupt: 

In [13]:
for i in range(10):
    print(gnn_sims[i], bert_base_sims[i], bert_finetuned_sims[i])

0.08416854010687934 0.8460969234048987 0.9983951589635919
0.016491794586181642 0.833133665058484 0.9976671605633305
0.23196786244710285 0.8470877222081871 0.9987822345481708
0.5839416927761502 0.8977293971430071 0.9992360580743953
-0.006031489372253418 0.8853157222698682 0.9986244353675016
0.12127556800842285 0.9402876451702362 0.998842757925823
0.0003962993621826172 0.8132934947169834 0.998260517956891
0.23954085857547514 0.7048886897545813 0.9975337280856543
0.3584686915079753 0.822692814923058 0.9982214676544215
0.7250172751290458 0.8272993250615464 0.997992663335089
